In [48]:
import MeCab
tokenizer = MeCab.Tagger("-Owakati")
import spacy
nlp = spacy.load('ja_ginza')


In [35]:
fname = '/home/iftekhar/amiebot/amieCore/amie_core/core/rasa/data/nlu/amie_nlu_ja.md'

In [36]:
def intent_chunks(fname):
    with open(fname, encoding='utf-8') as f:
        intents = f.read()
    return intents

In [37]:
file_contents = intent_chunks(fname).split('##')

In [261]:
for lines in file_contents:
    if lines:
        intents = lines.split('-')
        if(len(intents)) < 3 and lines.find('synonym') is -1:
            entities = extract_entities(tokenizer.parse(intents[1]))
            print(question_maker(entities))

['We providers are service 作りたい', 'providers We are service 開発を開始する場所', 'are providers We service どう', 'We providers are service 作りたい', 'We are providers service 始める方法', 'We providers are service 関する情報を教えてください', 'are providers We service どこ', 'providers are We service 入れられますか', 'are providers We service どう', 'providers We are service 何ができますか']
['Support Engineering Access workshops 作り方は', 'Access Engineering Support workshops 使うには', 'Access Support Engineering workshops 入れられますか', 'Support Engineering Access workshops 何ですか', 'Engineering Access Support workshops どうでしたか', 'Support Engineering Access workshops 開発を開始する場所', 'Support Engineering Access workshops 何ですか', 'Engineering Access Support workshops 使いたいです', 'Support Engineering Access workshops なにで', 'Access Engineering Support workshops どこですか']


In [246]:
def extract_entities(query):
    if not isinstance(query, list):
        query = query.split()
        
    entities = []
    for items in query:
        doc = nlp(items)
        for ent in doc.ents:
            if ent.text:
                entities.append(ent.text)
    entity = list(set(entities))
    if entity:
        return entity
    else:
        return query

In [304]:
import random

def question_maker(query):
    collector = []
    question = query
    count = 10
    delims = get_questions_delimiter_ja()
    while count > 0:
        shuffle_slice(question, 1, len(question))
        question_delimiter = random.choice(delims)
        collector.append(' '.join(question + [question_delimiter]))
        question = query
        count -=1
    return collector

In [305]:
question_maker('we are service providers'.split())

['we service providers are どこですか',
 'we providers are service 使い始める方法を教えてください',
 'we are providers service 使用しますか',
 'we are service providers どこですか',
 'we service providers are 使用しますか',
 'we providers service are 使用しますか',
 'we providers service are 作りたい',
 'we providers service are ついて教えてください',
 'we service providers are 始める方法',
 'we providers service are 何ができますか']

In [55]:
import os

def get_questions_delimiter_ja():
    #dir_path = os.path.dirname(os.path.realpath(__file__))
    dir_path = '/home/iftekhar/amiebot/amieCore/amie_core/core/tokenizer/stopwords'
    # questions_delimiter_ja = os.path.join(dir_path, 'stopwords', 'questions_delimiter.txt')
    questions_delimiter_ja = os.path.join(dir_path, 'questions_delimiter.txt')

    with open(questions_delimiter_ja, encoding='utf-8') as fr:
        questions_delimiter = fr.read().splitlines()

    return questions_delimiter


In [241]:
def shuffle_slice(a, start, stop):
    i = start
    while (i < stop-1):
        idx = random.randrange(i, stop)
        a[i], a[idx] = a[idx], a[i]
        i += 1

In [262]:
result = {'intent:what is mobicontrol': ['are providers service We 使うには', 'are service providers We 開発を開始する場所', 'are service providers We どこ', 'providers are service We 開発を開始する場所', 'service providers are We 何が', 'are service providers We 使い始めるには', 'service are providers We どの', 'providers are service We どうですか', 'service providers are We どちら', 'service providers are We ことは何ですか'], 'intent:what is SSAP': ['Support Access Engineering workshops 始める方法', 'Engineering Access Support workshops どこですか', 'Access Support Engineering workshops どちら', 'Access Engineering Support workshops どの', 'Engineering Support Access workshops どんな', 'Engineering Access Support workshops 何ですか', 'Support Access Engineering workshops 作りたい', 'Access Support Engineering workshops どの', 'Support Access Engineering workshops 入れられますか', 'Support Access Engineering workshops どの']}

In [285]:
for key, values in result.items():
    collector = ''
    for sentences in values:
        collector += sentences + '\n'
    print(collector)
    break

are providers service We 使うには
are service providers We 開発を開始する場所
are service providers We どこ
providers are service We 開発を開始する場所
service providers are We 何が
are service providers We 使い始めるには
service are providers We どの
providers are service We どうですか
service providers are We どちら
service providers are We ことは何ですか



In [273]:
import re

In [295]:
file_contents = intent_chunks(fname)
for lines, values in result.items():
    for match in re.finditer(lines, file_contents):
        collector = '\n'
        end_flag = len(values)
        for sentences in values:
            if end_flag > 1:
                collector += '- ' + sentences + '\n'
            else:
                collector += '- ' + sentences 
            end_flag -= 1
        file_contents = (file_contents[:match.end()] + collector + file_contents[match.end():])

In [ ]:
file_contents

In [297]:
file = open(fname,"w")
file.truncate(0)
file.write(file_contents)
file.close()